In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

spark

events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))
devices = spark.read.option("header", "true").csv("/home/iceberg/data/devices.csv")



24/11/29 23:41:18 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
events.show()
print((events.count(), len(events.columns)))
events.printSchema()

+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|
|  747494706| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-26 16:03:...|2021-09-26 00:00:00|
|  747494706| 532630305|    NULL|admin.zachwilson....|         /|2021-02-21 16:08:...|2021-02-21 00:00:00|
| -824540328| 532630305|    NULL|admi

In [8]:
devices.show()
print((devices.count(), len(devices.columns)))
devices.printSchema()

+-----------+--------------------+-------+------------------+
|  device_id|        browser_type|os_type|       device_type|
+-----------+--------------------+-------+------------------+
|-2147042689|             Firefox| Ubuntu|             Other|
|-2146219609|            WhatsApp|  Other|            Spider|
|-2145574618|       Chrome Mobile|Android|Generic Smartphone|
|-2144707350|Chrome Mobile Web...|Android|  Samsung SM-G988B|
|-2143813999|Mobile Safari UI/...|    iOS|            iPhone|
|-2142634982|   Chrome Mobile iOS|    iOS|            iPhone|
|-2142350383|   Chrome Mobile iOS|    iOS|            iPhone|
|-2141256237|   Chrome Mobile iOS|    iOS|            iPhone|
|-2138977887|             Firefox|OpenBSD|             Other|
|-2136667425|              Chrome|Windows|             Other|
|-2136444196|   Chrome Mobile iOS|    iOS|            iPhone|
|-2136415223|Chrome Mobile Web...|Android| Samsung SM-A305GT|
|-2136251094|             Firefox| Ubuntu|             Other|
|-213605

In [6]:
df = events.join(devices, events["device_id"] == devices["device_id"]) \
    .select(events["*"], devices["browser_type"], devices["os_type"], devices["device_type"])

df.show()

+-----------+----------+--------+--------------------+----------+--------------------+-------------------+------------+-------+-----------+
|    user_id| device_id|referrer|                host|       url|          event_time|         event_date|browser_type|os_type|device_type|
+-----------+----------+--------+--------------------+----------+--------------------+-------------------+------------+-------+-----------+
| 1037710827| 532630305|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|       Other|  Other|      Other|
|  925588856| 532630305|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|       Other|  Other|      Other|
|-1180485268| 532630305|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|       Other|  Other|      Other|
|-1044833855| 532630305|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|       Other|  Other|      Other|
|  747494706| 532630

In [9]:
df.printSchema()
print((df.count(), len(df.columns)))


root
 |-- user_id: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- referrer: string (nullable = true)
 |-- host: string (nullable = true)
 |-- url: string (nullable = true)
 |-- event_time: string (nullable = true)
 |-- event_date: timestamp (nullable = true)
 |-- browser_type: string (nullable = true)
 |-- os_type: string (nullable = true)
 |-- device_type: string (nullable = true)

(404813, 10)


In [11]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"), col("browser_type")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) 
sorted.explain()
sorted.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#374, event_date#29, browser_type#56, os_type#57, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, browser_type#56 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=471]
         +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56, os_type#57, device_type#58]
            +- BroadcastHashJoin [device_id#18], [device_id#55], Inner, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- Filter isnotnull(device_id#18)
               :     +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,

+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+------------+-------+------------------+
|    user_id|  device_id|            referrer|                host|                 url|          event_time|         event_date|browser_type|os_type|       device_type|
+-----------+-----------+--------------------+--------------------+--------------------+--------------------+-------------------+------------+-------+------------------+
| 1129583063|  532630305|                NULL|admin.zachwilson....|                   /|2021-01-07 09:21:...|2021-01-07 00:00:00|       Other|  Other|             Other|
|-1180485268|  532630305|                NULL|    www.eczachly.com|                   /|2021-01-07 18:45:...|2021-01-07 00:00:00|       Other|  Other|             Other|
| 1129583063|  532630305|                NULL|    www.eczachly.com|                   /|2021-01-07 21:57:...|2021-01-07 00:00:00|       Other|  Other|

In [12]:
global_sorted = df.repartition(10, col("event_date")) \
        .sort(col("event_date"), col("host"), col("browser_type")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) 
global_sorted.explain()
global_sorted.show()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#436, event_date#29, browser_type#56, os_type#57, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, browser_type#56 ASC NULLS FIRST], true, 0
      +- Exchange rangepartitioning(event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, browser_type#56 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=671]
         +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=669]
            +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56, os_type#57, device_type#58]
               +- BroadcastHashJoin [device_id#18], [device_id#55], Inner, BuildRight, false
                  :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp

In [13]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [14]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [15]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_type STRING,
    os_type STRING,
    device_type STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [16]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [17]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [20]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp"))
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_type"), col("host"))


start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [21]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files



size,num_files,sorted
5091334,4,sorted
5552944,4,unsorted
